In [4]:
from sympy import *
import numpy as np 
init_printing()

# Definitions
eta, dx, dz = symbols( 'eta, dx, dz' )
D11, D12, D13, D21, D22, D23 ,D31, D32, D33 = symbols('D11, D12, D13, D21, D22, D23 ,D31, D32, D33')
Exx, Ezz, Gxz = symbols('Exx, Ezz, Gxz')
stress_labels = ['Txx', 'Tzz', 'Txz']

D11 = 2*eta; D12 = 0*eta; D13 = 0*eta;
D21 = 0*eta; D22 = 2*eta; D23 = 0*eta;
D31 = 0*eta; D32 = 0*eta; D33 = 1*eta;

Dv = Matrix( [ [D11, D12, D13], [D21, D22, D23], [D31, D32, D33]  ] )
div = Exx+Ezz
Ed  = Matrix( [ [Exx-1/3*div], [Ezz-1/3*div], [Gxz] ] )
T  = Dv*Ed;
for i in range(len(stress_labels)):
    print(stress_labels[i] + ' = ' + ccode( T[i]) + ';' )


Txx = 2*eta*(0.666666666666667*Exx - 0.333333333333333*Ezz);
Tzz = 2*eta*(-0.333333333333333*Exx + 0.666666666666667*Ezz);
Txz = Gxz*eta;


In [14]:
# Compressibility switch
comp = symbols('comp')

# Stencil x
uW,uC,uE,uS,uN,vSW,vSE,vNW,vNE = symbols('uW,uC,uE,uS,uN,vSW,vSE,vNW,vNE')

inW,inE,inSu,inSv,inNu,inNv = symbols('inW,inE,inSu,inSv,inNu,inNv')

dofs = Matrix([uW,uC,uE,uS,uN,vSW,vSE,vNW,vNE])
#----------------------------------------#
divW = 1/dx*(uC-uW) + 1/dz*(vNW-vSW)
ExxW = 1/dx*(uC-uW) - comp*1/3*divW;
EzzW = 0;
GxzW = 0
EdW  = Matrix([[inW*ExxW], [inW*EzzW], [inW*GxzW]])
TW   = Dv*EdW
TxxW = TW[0].subs(eta,'etaW')
#TxxW = 2* etaW* ExxW
#----------------------------------------#
divE = 1/dx*(uE-uC) + 1/dz*(vNE-vSE)
ExxE = 1/dx*(uE-uC) - comp*1/3*divE
EzzE = 0;
GxzE = 0
EdE  = Matrix([[inE*ExxE], [inE*EzzE], [inE*GxzE]])
TE   = Dv*EdE
TxxE = TE[0].subs(eta,'etaE')
#----------------------------------------#
ExxS = 0
EzzS = 0
GxzS = inSu*1/dz*(uC-uS) + inSv*1/dx*(vSE-vSW)
EdS  = Matrix([[ExxS], [EzzS], [GxzS]])
TS   = Dv*EdS
TxzS = TS[2].subs(eta,'etaS')
#----------------------------------------#
ExxN = 0
EzzN = 0
GxzN = inNu*1/dz*(uN-uC) + inNv*1/dx*(vNE-vNW)
EdN  = Matrix([[ExxN], [EzzN], [GxzN]])
TN   = Dv*EdN
TxzN = TN[2].subs(eta,'etaN')
#----------------------------------------#
Fx = 1/dx*(TxxE - TxxW) + 1/dz*(TxzN - TxzS)  
# Fx = -Fx

for i in range(len(dofs)):
    cUc = Fx.diff(dofs[i])
    print(str(dofs[i]) + ' = ' +  ccode(cUc) + ';')

uW = -2*etaW*inW*((1.0L/3.0L)*comp/dx - 1/dx)/dx;
uC = (-etaN*inNu/dz - etaS*inSu/dz)/dz + (2*etaE*inE*((1.0L/3.0L)*comp/dx - 1/dx) - 2*etaW*inW*(-1.0L/3.0L*comp/dx + 1.0/dx))/dx;
uE = 2*etaE*inE*(-1.0L/3.0L*comp/dx + 1.0/dx)/dx;
uS = etaS*inSu/pow(dz, 2);
uN = etaN*inNu/pow(dz, 2);
vSW = -2.0L/3.0L*comp*etaW*inW/(dx*dz) + etaS*inSv/(dx*dz);
vSE = (2.0L/3.0L)*comp*etaE*inE/(dx*dz) - etaS*inSv/(dx*dz);
vNW = (2.0L/3.0L)*comp*etaW*inW/(dx*dz) - etaN*inNv/(dx*dz);
vNE = -2.0L/3.0L*comp*etaE*inE/(dx*dz) + etaN*inNv/(dx*dz);


In [18]:
# Compressibility switch
comp = symbols('comp')

# Stencil z
vW,vC,vE,vS,vN,uSW,uSE,uNW,uNE = symbols('vW,vC,vE,vS,vN,uSW,uSE,uNW,uNE')

inS,inN,inWu,inWv,inEu,inEv = symbols('inS,inN,inWu,inWv,inEu,inEv')

dofs = Matrix([vW,vC,vE,vS,vN,uSW,uSE,uNW,uNE])
#----------------------------------------#
divS = 1/dz*(vC-vS) + 1/dx*(uSE-uSW)
ExxS = 0
EzzS = 1/dz*(vC-vS) - comp*1/3*divS;
GxzS = 0
EdS  = Matrix([[inS*ExxS], [inS*EzzS], [inS*GxzS]])
TS   = Dv*EdS
TzzS = TS[1].subs(eta,'etaS')
#----------------------------------------#
divN = 1/dz*(vN-vC) + 1/dx*(uNE-uNW)
ExxN = 0
EzzN = 1/dz*(vN-vC)  - comp*1/3*divN;
GxzN = 0
EdN  = Matrix([[inN*ExxN], [inN*EzzN], [inN*GxzN]])
TN   = Dv*EdN
TzzN = TN[1].subs(eta,'etaN')
#----------------------------------------#
ExxW = 0
EzzW = 0
GxzW = inWu*1/dz*(uNW-uSW) + inWv*1/dx*(vC-vW)
EdW  = Matrix([[ExxW], [EzzW], [GxzW]])
TW   = Dv*EdW
TxzW = TW[2].subs(eta,'etaW')
# #----------------------------------------#
ExxE = 0
EzzE = 0
GxzE = inEu*1/dz*(uNE-uSE) + inEv*1/dx*(vE-vC)
EdE  = Matrix([[ExxE], [EzzE], [GxzE]])
TE   = Dv*EdE
TxzE = TE[2].subs(eta,'etaE')
#----------------------------------------#
Fz = 1/dz*(TzzN - TzzS) + 1/dx*(TxzE - TxzW)  
# Fz = -Fz
for i in range(9):
    cVc = Fz.diff(dofs[i])
    print(str(dofs[i]) + ' = ' +  ccode(cVc) + ';') #print(cUc)

vW = etaW*inWv/pow(dx, 2);
vC = (2*etaN*inN*((1.0L/3.0L)*comp/dz - 1/dz) - 2*etaS*inS*(-1.0L/3.0L*comp/dz + 1.0/dz))/dz + (-etaE*inEv/dx - etaW*inWv/dx)/dx;
vE = etaE*inEv/pow(dx, 2);
vS = -2*etaS*inS*((1.0L/3.0L)*comp/dz - 1/dz)/dz;
vN = 2*etaN*inN*(-1.0L/3.0L*comp/dz + 1.0/dz)/dz;
uSW = -2.0L/3.0L*comp*etaS*inS/(dx*dz) + etaW*inWu/(dx*dz);
uSE = (2.0L/3.0L)*comp*etaS*inS/(dx*dz) - etaE*inEu/(dx*dz);
uNW = (2.0L/3.0L)*comp*etaN*inN/(dx*dz) - etaW*inWu/(dx*dz);
uNE = -2.0L/3.0L*comp*etaN*inN/(dx*dz) + etaE*inEu/(dx*dz);
